## Pre-process_Nuu_Chah_Nulth_data

In [1]:
import os, gzip, pickle
import numpy as np
import pandas as pd
from collections import Counter
from selectiontest import selectiontest


path = "/Users/helmutsimon/Google Drive/Genetics/bayes_coalescent_est/data"
if not os.getcwd() == path:
    os.chdir(path)
    
pd.options.mode.chained_assignment = None

def calculate_sfs(n, sample_table, ancestral, counts):
    cum = np.cumsum(counts)
    num_sequences = counts.sum()
    indices = [np.argmax(cum>i) for i in range(num_sequences)]
    sample = sample_table.iloc[indices]
    occurrences = list()
    for column in sample:
        occurrences.append(sum(sample[column] != ancestral[column]))
    sfs_ = Counter(occurrences)
    sfs_ = [sfs_[i] for i in range(1, n)]
    return sfs_

Read csv file and extract data into tables

In [2]:
table = pd.read_csv("NuuChahNulthData_full.csv", header=None)
sample_table = table.iloc[2:]
counts = sample_table.iloc[:,-1]
counts = counts.reset_index(drop=True)      #counts is number of copies of context in sample
counts = counts.astype('int')
sample_table.drop(sample_table.columns[[-1,]], axis=1, inplace=True)
sample_table = sample_table.reset_index(drop=True)
#print(sample_table)                    
ancestral = table.iloc[0]           
#print(ancestral)
print(sample_table.head())

  0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17
0  A  G  G  A  A  T  C  C  T  C  T  T  C  T  C  T  T  C
1  A  G  G  A  A  T  C  C  T  T  T  T  C  T  C  T  T  C
2  G  A  G  G  A  C  C  C  T  C  T  T  C  C  C  T  T  T
3  G  G  A  G  A  C  C  C  C  C  T  T  C  C  C  T  T  C
4  G  G  G  A  A  T  C  C  T  C  T  T  C  T  C  T  T  C


Take SFS of a random sample. Use Wright Fisher prior information based on values for effective popu;ation size and mutation rate in Griffith and Tavare. Put in form for run_bayesian_model.py

In [4]:
n = 55
sfs = calculate_sfs(n, sample_table, ancestral, counts)
print(sfs)
results_list = [sfs, None, None, None, None] 
with gzip.open('sample_params_ncn55_00', 'wb') as outfile:
    pickle.dump(results_list, outfile)
print('Number of segregating sites ', np.sum(sfs))
N = 600
print('Expected Wright-Fisher TMRCA', 2 * N * (1 - (1/n)))
print('Expected Wright-Fisher ttl  ', np.sum(2 * N / np.arange(1, n)))
print("Tajima's D                  ", selectiontest.calculate_D(sfs))
seq_mut_rate = 4e-3   # Griffiths  andTavare
n_seq = np.arange(1, n)
thom = np.sum(sfs * n_seq) / (n * seq_mut_rate)
print("Thomson estimate TMRCA      ", thom)

[5, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Number of segregating sites  18
Expected Wright-Fisher TMRCA 1178.1818181818182
Expected Wright-Fisher ttl   5490.516472493125
Tajima's D                   -0.5020156500630947
Thomson estimate TMRCA       595.4545454545455
